In [1]:
from pyspark import SparkConf, SparkContext
import os
os.environ['PYSPARK_PYTHON'] = '/Library/Frameworks/Python.framework/Versions/3.6/bin/python3'

from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local").appName("streaming").getOrCreate()

#sc.stop()
conf = SparkConf().setMaster("local").setAppName("streaming")
sc = SparkContext.getOrCreate()

In [3]:
static = spark.read.json("data/activity-data/")
dataSchema = static.schema

In [5]:
static.take(1)

[Row(Arrival_Time=1424686735090, Creation_Time=1424686733090638193, Device='nexus4_1', Index=18, Model='nexus4', User='g', gt='stand', x=0.0003356934, y=-0.0005645752, z=-0.018814087)]

In [9]:
## streaming version

streaming = spark.readStream.schema(dataSchema)\
.option("maxFilesPerTrigger", 1)\
.json("data/activity-data")

activityCounts = streaming.groupBy("gt").count()

activityQuery = activityCounts.writeStream \
.queryName("activity_counts_222")\
.format("memory")\
.outputMode("complete")\
.start()

from time import sleep
for x in range(5):
    spark.sql("SELECT * FROM activity_counts_222").show()
    sleep(1)

activityQuery.awaitTermination()



+---+-----+
| gt|count|
+---+-----+
+---+-----+

+---+-----+
| gt|count|
+---+-----+
+---+-----+

+----------+-----+
|        gt|count|
+----------+-----+
|  stairsup|10461|
|       sit|12308|
|     stand|11384|
|      walk|13255|
|      bike|10797|
|stairsdown| 9363|
|      null|10446|
+----------+-----+

+----------+-----+
|        gt|count|
+----------+-----+
|  stairsup|10461|
|       sit|12308|
|     stand|11384|
|      walk|13255|
|      bike|10797|
|stairsdown| 9363|
|      null|10446|
+----------+-----+

+----------+-----+
|        gt|count|
+----------+-----+
|  stairsup|20913|
|       sit|24617|
|     stand|22769|
|      walk|26511|
|      bike|21594|
|stairsdown|18728|
|      null|20894|
+----------+-----+



StreamingQueryException: 'Job aborted due to stage failure: Task 36 in stage 330.0 failed 1 times, most recent failure: Lost task 36.0 in stage 330.0 (TID 32187, localhost, executor driver): ExecutorLostFailure (executor driver exited caused by one of the running tasks) Reason: Executor heartbeat timed out after 644615 ms\nDriver stacktrace:\n=== Streaming Query ===\nIdentifier: activity_counts_222 [id = e69056be-0a36-4f31-8a45-c4365dd7b037, runId = 995760ac-dfa8-4c49-ad0c-c9b382052556]\nCurrent Committed Offsets: {FileStreamSource[file:/Users/aakash/training/spark/data/activity-data]: {"logOffset":78}}\nCurrent Available Offsets: {FileStreamSource[file:/Users/aakash/training/spark/data/activity-data]: {"logOffset":79}}\n\nCurrent State: ACTIVE\nThread State: RUNNABLE\n\nLogical Plan:\nAggregate [gt#78], [gt#78, count(1) AS count#221L]\n+- StreamingExecutionRelation FileStreamSource[file:/Users/aakash/training/spark/data/activity-data], [Arrival_Time#72L, Creation_Time#73L, Device#74, 

In [ ]:
deviceModelStats = streaming.cube("gt", "model").avg()\
.drop("avg(Arrival_time)")\
.drop("avg(Creation_Time)")\
.drop("avg(Index)")\
.writeStream.queryName("device_counts_1").format("memory")\
.outputMode("complete")\
.start()


from time import sleep
for x in range(5):
    spark.sql("SELECT * FROM device_counts_1").show()
    sleep(1)

deviceModelStats.awaitTermination()

+---+-----+------+------+------+
| gt|model|avg(x)|avg(y)|avg(z)|
+---+-----+------+------+------+
+---+-----+------+------+------+

+---+-----+------+------+------+
| gt|model|avg(x)|avg(y)|avg(z)|
+---+-----+------+------+------+
+---+-----+------+------+------+

+----------+------+--------------------+--------------------+--------------------+
|        gt| model|              avg(x)|              avg(y)|              avg(z)|
+----------+------+--------------------+--------------------+--------------------+
|      null|nexus4|-0.00534777761728...|-0.00471625131602...|0.001053548924143...|
|      null|nexus4|0.001243822064214...|-0.00508318886798...|-0.01029860468302...|
|      null|  null|0.001243822064214...|-0.00508318886798...|-0.01029860468302...|
|      bike|nexus4| 0.02652570661093831| -0.0112163392482819|-0.08351623094110396|
|     stand|  null|-1.91564553417079...|-1.20081210216091...|-6.67507354181460...|
|       sit|nexus4|-5.48643225000002...|-1.75231850243742...|-2.212524